### Web scraping exercises - Module Section 10.3.3

In [1]:
# imports
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
# set executable path and the URL for scraping
# Be sure to note Terms of Use for websites to see if automated browsing is permitted. Automated browsing can overload websites
# and some websites prohibit it.
executable_path={'executable_path': ChromeDriverManager().install()}
browser=Browser('chrome', **executable_path, headless=False)  # creates the automated browser window



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\sarah\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [3]:
#Visit the mars NASA news site
url='https://redplanetscience.com'
browser.visit(url)
# Search for elmts with tag 'div' and attribute 'list_text'. Inspected website HTML and seeing div class="list_text"
# Optional delay for loading the page; wait 1 sec before loading, usually helpful if dynamic, image-heavy website.
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [4]:
html=browser.html
news_soup=soup(html, 'html.parser')
# div.list_text looks for div tag with list_text class. Then slide_elem captures a chunk of the html with this class.
slide_elem=news_soup.select_one('div.list_text') 

In [5]:
# Sample statements to see what printed:

# slide_elem.find('div', class_='content_title')  
# output is the following line: (find the specific title within this div tag)
# <div class="content_title">NASA's InSight Flexes Its Arm While Its 'Mole' Hits Pause</div>

# print(slide_elem)
# output is the following lines:
# <div class="list_text">
# <div class="list_date">June 21, 2021</div>
# <div class="content_title">NASA's InSight Flexes Its Arm While Its 'Mole' Hits Pause</div>
# <div class="article_teaser_body">Now that the lander's robotic arm has helped the mole get underground, it will resume science activities that have been on hold.</div>
# </div>

# print(news_soup)
# output is what looks like all the html text for the webpage, with divs, navs, etc.

In [6]:
slide_elem.find('div', class_='content_title')

<div class="content_title">NASA's MAVEN Explores Mars to Understand Radio Interference at Earth</div>

In [7]:
# Want to get just the text. Use get_text method to pull out the text of the element.
news_title=slide_elem.find('div', class_='content_title').get_text()
news_title

"NASA's MAVEN Explores Mars to Understand Radio Interference at Earth"

In [8]:
# Pull the paragraph text from the summary under the title on the webpage.
# the .find() method from BeautifulSoup gives first class and attribute found
# the .find_all() method gives all the sections with the given class and attribute

# news_p=slide_elem.find_all('div', class_='article_teaser_body')
# output is:
# [<div class="article_teaser_body">The NASA rover touched down eight years ago, on Aug. 5, 2012, 
#  and will soon be joined by a second rover, Perseverance.</div>]


news_p=slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'NASA’s MAVEN spacecraft has discovered “layers” and “rifts” in the electrically charged part of the upper atmosphere of Mars.'

### Featured Images - Module Section 10.3.4

In [9]:
# Visit URL
url='https://spaceimages-mars.com/'
browser.visit(url)

In [10]:
# Inspect with DevTools to see that can use button tag.
# Find and click the full image button. It is the 2nd of 3 button tags on the page so index 1.
full_image_elem=browser.find_by_tag('button')[1]  # this is: <splinter.driver.webdriver.WebDriverElement at 0x22708e49608>
full_image_elem.click()

In [11]:
# Parse the resulting html with soup
html=browser.html
img_soup=soup(html, 'html.parser') # this is the html from the image that pops up when clicking 'full image' button

In [12]:
# Inspect the html for the image to find tag and class for the actual image.
# Find the relative image url. img tag with class fancybox-image. The get('src') pulls the link to the image.
img_url_rel=img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars1.jpg'

In [13]:
# The above is only a partial link. Need to add on how to get to the page itself.
# Utilize f string to get formatting but include the url as it updates.
img_url=f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars1.jpg'

### Scraping Tables - Module Section 10.3.5

In [14]:
# Scrape the entire table with Pandas read_html function
df=pd.read_html('https://galaxyfacts-mars.com/')[0] #index[0] pulls only first table it encounters
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [15]:
# convert table back into html
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [16]:
# End automated browser session.
browser.quit()